In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from IPython.display import Image
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from seaborn import plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

import dateutil.parser

# enables inline plots, without it plots don't show up in the notebook
%matplotlib inline

In [5]:
import pickle
with open("0130_private_prop_dict.pkl", 'rb') as picklefile: 
    priv_dict = pickle.load(picklefile)
print(priv_dict)

{2411722240007: {'building_market_value': '$80,000', 'homestead_mrkt_val_excl': '$22,030', 'property_type': 'RESIDENTIAL', 'total_market_value': '$169,000', 'owner_name': 'MARY JANE GRAPER', 'total_tax': '', 'est_market_value': '$169,000', 'homestead_status': 'HOMESTEAD', 'tax_market_value': '$146,970', 'total_net_tax': '$2,524.34', 'property_id': 2411722240007, 'construction_year': '1922', 'address': '222 9th St S', 'land_market_value': '$89,000'}, 1602924210184: {'building_market_value': '', 'homestead_mrkt_val_excl': '', 'property_type': 'COMMERCIAL PREFERRED', 'total_market_value': '', 'owner_name': 'BOYS & GIRLS CLUB OF TC', 'total_tax': '', 'est_market_value': '', 'homestead_status': 'NON-HOMESTEAD', 'tax_market_value': '', 'total_net_tax': '', 'property_id': '16-029-24-21-0184', 'construction_year': '1968', 'address': '2410 Irving Ave N', 'land_market_value': ''}, 2202924140554: {'building_market_value': '$298,200', 'homestead_mrkt_val_excl': '', 'property_type': 'COMMERCIAL NON

In [8]:
priv_df = pd.DataFrame.from_dict(priv_dict, orient="index")

In [10]:
priv_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
building_market_value      696 non-null object
homestead_mrkt_val_excl    696 non-null object
property_type              696 non-null object
total_market_value         696 non-null object
owner_name                 696 non-null object
total_tax                  696 non-null object
est_market_value           696 non-null object
homestead_status           696 non-null object
tax_market_value           696 non-null object
total_net_tax              696 non-null object
property_id                696 non-null object
construction_year          696 non-null object
address                    696 non-null object
land_market_value          696 non-null object
sale_price                 269 non-null object
sale_date                  269 non-null object
transaction_type           269 non-null object
dtypes: object(17)
memory usage: 97.9+ KB


In [11]:
priv_df.head()

,building_market_value,homestead_mrkt_val_excl,property_type,total_market_value,owner_name,total_tax,est_market_value,homestead_status,tax_market_value,total_net_tax,property_id,construction_year,address,land_market_value,sale_price,sale_date,transaction_type
102824110123,,,COMMERCIAL PREFERRED,,HOLY TRINITY LUTH CH MPLS,,,NON-HOMESTEAD,,,01-028-24-11-0123,1923,2730 31St St E,,NaN,NaN,NaN
302824210283,,,COMMERCIAL PREFERRED,,METRO TRANSIT COMMISSION,,,NON-HOMESTEAD,,,03-028-24-21-0283,1989,3106 Nicollet Ave,,NaN,NaN,NaN
302824240197,,,COMMERCIAL PREFERRED,,YOUNG MENS CHRISTIAN ASN T/C,,,NON-HOMESTEAD,,,03-028-24-24-0197,1961,3335 Blaisdell Ave,,NaN,NaN,NaN
402824210016,"$3,195,600",,COMMERCIAL PREFERRED,"$4,050,000",SONS OF NORWAY,,"$4,050,000",NON-HOMESTEAD,"$4,050,000","$161,031.14",04-028-24-21-0016,1961,1455 Lake St W,"$854,400",NaN,NaN,NaN
502824120005,"$16,349,900",,COMMERCIAL PREFERRED,"$22,720,000",ACKY-CALHOUN LLC,,"$22,820,000",NON-HOMESTEAD,"$22,820,000","$913,934.73",05-028-24-12-0005,1955,3033 Excelsior Blvd,"$6,370,100","$24,500,000","August, 2000",Warranty Deed


In [26]:
# Is this an appropriate/best way to deal with blank values?

def replace_empty_values(df):
    df_cl = df.replace(r'\s+', np.nan, regex=True)
    df_cl = df.replace('', np.nan)
    return df_cl

priv_df_cl = replace_empty_values(priv_df)

,building_market_value,homestead_mrkt_val_excl,property_type,total_market_value,owner_name,total_tax,est_market_value,homestead_status,tax_market_value,total_net_tax,property_id,construction_year,address,land_market_value,sale_price,sale_date,transaction_type
102824110123,NaN,NaN,COMMERCIAL PREFERRED,NaN,HOLY TRINITY LUTH CH MPLS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,01-028-24-11-0123,1923,2730 31St St E,NaN,NaN,NaN,NaN
302824210283,NaN,NaN,COMMERCIAL PREFERRED,NaN,METRO TRANSIT COMMISSION,NaN,NaN,NON-HOMESTEAD,NaN,NaN,03-028-24-21-0283,1989,3106 Nicollet Ave,NaN,NaN,NaN,NaN
302824240197,NaN,NaN,COMMERCIAL PREFERRED,NaN,YOUNG MENS CHRISTIAN ASN T/C,NaN,NaN,NON-HOMESTEAD,NaN,NaN,03-028-24-24-0197,1961,3335 Blaisdell Ave,NaN,NaN,NaN,NaN
402824210016,"$3,195,600",NaN,COMMERCIAL PREFERRED,"$4,050,000",SONS OF NORWAY,NaN,"$4,050,000",NON-HOMESTEAD,"$4,050,000","$161,031.14",04-028-24-21-0016,1961,1455 Lake St W,"$854,400",NaN,NaN,NaN
502824120005,"$16,349,900",NaN,COMMERCIAL PREFERRED,"$22,720,000",ACKY-CALHOUN LLC,NaN,"$22,820,000",NON-HOMESTEAD,"$22,820,000","$913,934.73",05-028-24-12-0005,1955,3033 Excelsior Blvd,"$6,370,100","$24,500,000","August, 2000",Warranty Deed
502924130012,NaN,NaN,LAND COMMERCIAL PREFERRED,"$180,000",MCA HOLDINGS LLC,NaN,"$180,000",NON-HOMESTEAD,"$180,000","$5,765.46",05-029-24-13-0012,1950,4021 Thomas Ave N,"$180,000",NaN,NaN,NaN
511722320050,"$166,000",NaN,RESIDENTIAL TWO UNIT,"$367,000",AM PARTNERS LLC,NaN,"$367,000",NON-HOMESTEAD,"$367,000","$5,756.66",05-117-22-32-0050,1966,1101 Lasalle Ave,"$201,000","$450,000","April, 2005",Warranty Deed
1002824220092,NaN,NaN,COMMERCIAL PREFERRED,NaN,SAGRADO CORAZON DE JESUS,NaN,NaN,NON-HOMESTEAD,NaN,NaN,10-028-24-22-0092,1934,3800 Pleasant Ave,NaN,NaN,NaN,NaN
1102924140067,"$136,800",NaN,RESIDENTIAL TWO UNIT,"$149,000",NORDEAST PARTNERS LLC,NaN,"$149,000",NON-HOMESTEAD,"$149,000","$2,881.82",11-029-24-14-0067,1900,910 26Th St E,"$12,200","$166,500","July, 2015",Warranty Deed
1102924210015,"$1,254,900",NaN,INDUSTRIAL PREFERRED,"$1,995,000",FULLER AVE ESTATES LLC ETAL,NaN,"$1,995,000",NON-HOMESTEAD,"$1,995,000","$78,658.88",11-029-24-21-0015,1946,2701 University SE,"$740,100",NaN,NaN,NaN


In [36]:
def clean_scrape_data(df):
    df['building_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['homestead_mrkt_val_excl'].replace('[\$,)]','', regex=True).astype(float)
    df['total_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['total_tax'].replace('[\$,)]','', regex=True).astype(float)
    df['est_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['land_market_value'].replace('[\$,)]','', regex=True).astype(float)
    df['sale_price'].replace('[\$,)]','', regex=True).astype(float)

    # priv_df_cl['construction_year'].astype(int)

    # parse_time = lambda x: dateutil.parser.parse(x, dayfirst=False)
    # priv_df_cl['sale_date_object'] = priv_df['sale_date'].apply(parse_time)

clean_scrape_data(priv_df_cl)
    
priv_df_cl.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 696 entries, 102824110123 to 3602924430088
Data columns (total 17 columns):
building_market_value      610 non-null object
homestead_mrkt_val_excl    149 non-null object
property_type              696 non-null object
total_market_value         628 non-null object
owner_name                 696 non-null object
total_tax                  0 non-null float64
est_market_value           628 non-null object
homestead_status           696 non-null object
tax_market_value           628 non-null object
total_net_tax              628 non-null object
property_id                696 non-null object
construction_year          664 non-null object
address                    696 non-null object
land_market_value          628 non-null object
sale_price                 269 non-null object
sale_date                  269 non-null object
transaction_type           244 non-null object
dtypes: float64(1), object(16)
memory usage: 97.9+ KB


In [40]:
priv_df.head()

,building_market_value,homestead_mrkt_val_excl,property_type,total_market_value,owner_name,total_tax,est_market_value,homestead_status,tax_market_value,total_net_tax,property_id,construction_year,address,land_market_value,sale_price,sale_date,transaction_type
102824110123,,,COMMERCIAL PREFERRED,,HOLY TRINITY LUTH CH MPLS,,,NON-HOMESTEAD,,,01-028-24-11-0123,1923,2730 31St St E,,NaN,NaN,NaN
302824210283,,,COMMERCIAL PREFERRED,,METRO TRANSIT COMMISSION,,,NON-HOMESTEAD,,,03-028-24-21-0283,1989,3106 Nicollet Ave,,NaN,NaN,NaN
302824240197,,,COMMERCIAL PREFERRED,,YOUNG MENS CHRISTIAN ASN T/C,,,NON-HOMESTEAD,,,03-028-24-24-0197,1961,3335 Blaisdell Ave,,NaN,NaN,NaN
402824210016,"$3,195,600",,COMMERCIAL PREFERRED,"$4,050,000",SONS OF NORWAY,,"$4,050,000",NON-HOMESTEAD,"$4,050,000","$161,031.14",04-028-24-21-0016,1961,1455 Lake St W,"$854,400",NaN,NaN,NaN
502824120005,"$16,349,900",,COMMERCIAL PREFERRED,"$22,720,000",ACKY-CALHOUN LLC,,"$22,820,000",NON-HOMESTEAD,"$22,820,000","$913,934.73",05-028-24-12-0005,1955,3033 Excelsior Blvd,"$6,370,100","$24,500,000","August, 2000",Warranty Deed


In [ ]:
priv_df_cl.group_by("Address").

In [37]:
df = pd.read_csv('../data/raw/wcmsp-185935.csv')

In [38]:
cols = ['address', 'postal_code', 'energy_star_score', 'primary_property_type',
       'building_floor_area', 'parking_floor_area',
       'year_built', 'total_ghg_emissions',
       'site_eui', 'weather_normalized_site_eui',
       'source_eui', 'weather_normalized_source_eui',
       'water_use', 'unnamed']
df.columns = cols
df.head()

,address,postal_code,energy_star_score,primary_property_type,building_floor_area,parking_floor_area,year_built,total_ghg_emissions,site_eui,weather_normalized_site_eui,source_eui,weather_normalized_source_eui,water_use,unnamed
0,201 Chicago Ave S,55415,36,Office,62400,0,2012,971,115,118,251,252,708,NaN
1,1011 Nicollet Mall,55403,68,Office,59479,0,1920,986,90,93,256,266,882,NaN
2,4701 Beard Ave S,55410,0,Other,59172,0,1914,1622,157,160,427,428,163,NaN
3,8 60Th St W,55419,76,Worship Facility,60640,62000,1959,268,50,54,77,81,674,NaN
4,700 10Th Ave S,55415,89,Office,60528,0,1920,504,74,77,136,140,414,NaN


In [39]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 14 columns):
address                          257 non-null object
postal_code                      257 non-null object
energy_star_score                257 non-null int64
primary_property_type            257 non-null object
building_floor_area              257 non-null int64
parking_floor_area               257 non-null int64
year_built                       257 non-null int64
total_ghg_emissions              257 non-null int64
site_eui                         257 non-null int64
weather_normalized_site_eui      257 non-null int64
source_eui                       257 non-null int64
weather_normalized_source_eui    257 non-null int64
water_use                        257 non-null int64
unnamed                          0 non-null float64
dtypes: float64(1), int64(10), object(3)
memory usage: 28.2+ KB
